Carga y Preparación del Dataset


In [1]:
import pandas as pd

# Define la ruta del archivo.
file_path = r'C:\Users\USER\Documents\repositorios\Proyecto_Final_MLOps_Yesid\data\raw\statlog+german+credit+data\german.data'

# Lista de los nombres de las columnas en el orden correcto
column_names = [
    'estado_cuenta_actual', 'duracion_credito_meses', 'historial_credito',
    'proposito_credito', 'monto_credito', 'ahorros_o_bonos',
    'tiempo_empleo', 'tasa_de_cuotas', 'estado_civil_y_sexo',
    'otros_deudores_o_garantes', 'tiempo_residencia_actual', 'propiedad',
    'edad', 'otros_planes_cuotas', 'vivienda', 'numero_creditos_existentes',
    'trabajo', 'numero_personas_dependientes', 'telefono',
    'trabajador_extranjero', 'clase'
]

df = pd.read_csv(file_path, sep=' ', header=None)

df.columns = column_names

print(df.head())

  estado_cuenta_actual  duracion_credito_meses historial_credito  \
0                  A11                       6               A34   
1                  A12                      48               A32   
2                  A14                      12               A34   
3                  A11                      42               A32   
4                  A11                      24               A33   

  proposito_credito  monto_credito ahorros_o_bonos tiempo_empleo  \
0               A43           1169             A65           A75   
1               A43           5951             A61           A73   
2               A46           2096             A61           A74   
3               A42           7882             A61           A74   
4               A40           4870             A61           A73   

   tasa_de_cuotas estado_civil_y_sexo otros_deudores_o_garantes  ...  \
0               4                 A93                      A101  ...   
1               2                 A92 

Análisis de la Variable Objetivo

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   estado_cuenta_actual          1000 non-null   object
 1   duracion_credito_meses        1000 non-null   int64 
 2   historial_credito             1000 non-null   object
 3   proposito_credito             1000 non-null   object
 4   monto_credito                 1000 non-null   int64 
 5   ahorros_o_bonos               1000 non-null   object
 6   tiempo_empleo                 1000 non-null   object
 7   tasa_de_cuotas                1000 non-null   int64 
 8   estado_civil_y_sexo           1000 non-null   object
 9   otros_deudores_o_garantes     1000 non-null   object
 10  tiempo_residencia_actual      1000 non-null   int64 
 11  propiedad                     1000 non-null   object
 12  edad                          1000 non-null   int64 
 13  otros_planes_cuotas

In [3]:
df.describe()

,duracion_credito_meses,monto_credito,tasa_de_cuotas,tiempo_residencia_actual,edad,numero_creditos_existentes,numero_personas_dependientes,clase
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000,1.300000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086,0.458487
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000,2.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000,2.000000


In [4]:
df['proposito_credito'].value_counts()

proposito_credito
A43     280
A40     234
A42     181
A41     103
A49      97
A46      50
A45      22
A44      12
A410     12
A48       9
Name: count, dtype: int64

In [5]:
df['clase'].value_counts()

clase
1    700
2    300
Name: count, dtype: int64

Codificación de Variables Categóricas y Balanceo de Datos con SMOTE

In [6]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# características (X)  variable objetivo (y)
X = df.drop('clase', axis=1)
y = df['clase']

# Identificar variables categóricas

print(X.head())

  estado_cuenta_actual  duracion_credito_meses historial_credito  \
0                  A11                       6               A34   
1                  A12                      48               A32   
2                  A14                      12               A34   
3                  A11                      42               A32   
4                  A11                      24               A33   

  proposito_credito  monto_credito ahorros_o_bonos tiempo_empleo  \
0               A43           1169             A65           A75   
1               A43           5951             A61           A73   
2               A46           2096             A61           A74   
3               A42           7882             A61           A74   
4               A40           4870             A61           A73   

   tasa_de_cuotas estado_civil_y_sexo otros_deudores_o_garantes  \
0               4                 A93                      A101   
1               2                 A92           

In [7]:

# 1. Separar los datos en características (X) y la variable objetivo (y)
X = df.drop(columns=['clase']) # Todas las columnas excepto la clase
y = df['clase'] # Solo la columna 'clase'

# 2. Identificar y codificar variables categóricas

X_encoded = pd.get_dummies(X, drop_first=True)

# 3. Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_encoded, y)

# 4. Verificar la nueva distribución de la variable objetivo
print("Distribución original de la clase:")
print(y.value_counts())
print("\nDistribución de la clase después de SMOTE:")
print(y_res.value_counts())

Distribución original de la clase:
clase
1    700
2    300
Name: count, dtype: int64

Distribución de la clase después de SMOTE:
clase
1    700
2    700
Name: count, dtype: int64


División del Conjunto de Datos

In [8]:

# Dividir el conjunto de datos

# X_res y y_res son los datos ya balanceados
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42
)


print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)
print("Dimensiones de y_train:", y_train.shape)
print("Dimensiones de y_test:", y_test.shape)

Dimensiones de X_train: (1120, 48)
Dimensiones de X_test: (280, 48)
Dimensiones de y_train: (1120,)
Dimensiones de y_test: (280,)
